# Run Prerequirements

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
%run '/content/drive/MyDrive/nic_project/methods.ipynb'

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
/content/drive/MyDrive/nic_project/packages
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 721.3/721.3 kB 13.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 65.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 823.6/823.6 kB 69.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 68.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 731.7/731.7 MB 2.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 410.6/410.6 MB 2.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.6/121.6 MB 8.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.5/56.5 MB 10.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 124.2/124.2 MB 8.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 196.0/196.0 MB 6.1 MB/s eta 0:00:00
     ━━━

# Run the system on a single image

In [3]:
def upload_file():
  %cd /content/drive/MyDrive/nic_project/uploading_images
  uploaded = files.upload()
  file_path = None
  for name, data in uploaded.items():
      file_path = os.path.join('/content/drive/MyDrive/nic_project/uploading_images/', name)
  return file_path

file_path = upload_file()

def display_image_and_results(image_path, results_id, results_names):
  img = cv2.imread(image_path)
  img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

  fig, ax = plt.subplots(1, 2, figsize=(12, 6))

  ax[0].imshow(img)
  ax[0].axis('off')

  ax[1].text(0.1, 0.9, f"ID Number: {results_id['id_number']}", fontsize=12, ha='left', va='center')
  ax[1].text(0.1, 0.8, f"Name: {results_names['name1']} {results_names['name2']} {results_names['name3']}", fontsize=12, ha='left', va='center')
  ax[1].text(0.1, 0.7, f"Gender: {results_id['gender']}", fontsize=12, ha='left', va='center')
  ax[1].text(0.1, 0.6, f"Date of Birth: {results_id['bod']}", fontsize=12, ha='left', va='center')
  ax[1].axis('off')

  fig.suptitle("Sri Lankan National ID Information Extractor", fontsize=16, ha='center')
  plt.show()

def process_single_image(file_path):
  crop_image = get_transform(file_path)

  if crop_image is not None:
    crop_image = Image.fromarray(crop_image)
    results_id = get_text(crop_image, model_name='idtextdetect')
    results_names = get_text(crop_image, model_name='detectname')

    # Post-processings
    id_number = results_id['id_number']
    cleaned_id_number = ''.join(filter(str.isdigit, id_number))  # Keep only digits
    results_id['id_number'] = cleaned_id_number

    for key in ['name1', 'name2', 'name3']:
      results_names[key] = results_names[key].replace("'", '')
      results_names[key] = results_names[key].replace(';', '')
      results_names[key] = results_names[key].replace('‘', '')
      results_names[key] = results_names[key].replace('', '')
      results_names[key] = results_names[key].replace('!', 'I')
      results_names[key] = results_names[key].replace('\n', ' ')

    for key in ['name1', 'name2', 'name3']:
      results_names[key] = results_names[key].upper()

    for key in ['gender', 'bod']:
      results_id[key] = results_id[key].replace("'", '')
      results_id[key] = results_id[key].replace(';', '')
      results_id[key] = results_id[key].replace('‘', '')
      results_id[key] = results_id[key].replace('', '')
      results_id[key] = results_id[key].replace('\n', ' ')

     #Handle gender prediction
      gender_prediction = results_id['gender'].strip().upper()  # Convert to uppercase and remove leading/trailing whitespaces
      if gender_prediction.startswith('M'):
        results_id['gender'] = 'Male'
      elif gender_prediction.startswith('F'):
        results_id['gender'] = 'Female'
      else:
        results_id['gender'] = ''

    display_image_and_results(file_path, results_id, results_names)

  else:
    print("\n----------ERROR-----------")
    print("CORNER DETECTION FAILED !!!")

process_single_image(file_path)

/content/drive/MyDrive/nic_project/uploading_images


Saving 0001.jpg to 0001.jpg

0: 640x384 1 top_left, 1 top_right, 1 bottom_right, 1 bottom_left, 245.8ms
Speed: 12.3ms preprocess, 245.8ms inference, 1.4ms postprocess per image at shape (1, 3, 640, 384)

0: 480x640 1 id_number, 1 gender, 1 bod, 185.4ms
Speed: 1.7ms preprocess, 185.4ms inference, 0.8ms postprocess per image at shape (1, 3, 480, 640)
[325  91 493 122]
Loads checkpoint by http backend from path: https://download.openmmlab.com/mmocr/textrecog/satrn/satrn_shallow_5e_st_mj/satrn_shallow_5e_st_mj_20220915_152443-5fd04a4c.pth


Downloading: "https://download.openmmlab.com/mmocr/textrecog/satrn/satrn_shallow_5e_st_mj/satrn_shallow_5e_st_mj_20220915_152443-5fd04a4c.pth" to /root/.cache/torch/hub/checkpoints/satrn_shallow_5e_st_mj_20220915_152443-5fd04a4c.pth


KeyboardInterrupt: 

# Run the system on a directory(multiple images)

In [ ]:
# Main script
link = input("Enter the link to the images (e.g., /content/drive/MyDrive/nic_project/test_images): ")
dirs = os.listdir(link)

successful_corner_detections = 0
failed_corner_detections = 0

# Open a CSV file for writing
csv_file_path = '/content/drive/MyDrive/nic_project/results/results.csv'
with open(csv_file_path, 'w', newline='', encoding='utf-8') as csvfile:
    fieldnames = ['Image', 'ID Number', 'Name', 'Gender', 'Date of Birth']
    writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
    writer.writeheader()

    for dir in dirs:
      path = os.path.join(link, dir)
      crop_image = get_transform(path)

      if crop_image is not None:
        crop_image = Image.fromarray(crop_image)
        results_id = get_text(crop_image, model_name='idtextdetect')
        results_names = get_text(crop_image, model_name='detectname')

        # Post-processings
        id_number = results_id['id_number']
        cleaned_id_number = ''.join(filter(str.isdigit, id_number))  # Keep only digits
        results_id['id_number'] = cleaned_id_number

        for key in ['name1','name2','name3']:
          results_names[key] = results_names[key].replace("'", '')
          results_names[key] = results_names[key].replace(';', '')
          results_names[key] = results_names[key].replace('‘', '')
          results_names[key] = results_names[key].replace('', '')
          results_names[key] = results_names[key].replace('!', 'I')
          results_names[key] = results_names[key].replace('\n', ' ')

        for key in ['name1','name2','name3']:
          results_names[key] = results_names[key].upper()

        for key in ['gender', 'bod']:
          results_id[key] = results_id[key].replace("'", '')
          results_id[key] = results_id[key].replace(';', '')
          results_id[key] = results_id[key].replace('‘', '')
          results_id[key] = results_id[key].replace('', '')
          results_id[key] = results_id[key].replace('\n', ' ')

          # Handle gender prediction
        gender_prediction = results_id['gender'].strip().upper() # Convert to uppercase and remove leading/trailing whitespaces
        if gender_prediction.startswith('M'):
          results_id['gender'] = 'Male'
        elif gender_prediction.startswith('F'):
          results_id['gender'] = 'Female'
        else:
          results_id['gender'] = ''


        writer.writerow({
            'Image': dir,
            'ID Number': results_id['id_number'],
            'Name': results_names['name1']+' '+results_names['name2']+' '+results_names['name3'],
            'Gender': results_id['gender'],
            'Date of Birth': results_id['bod']
            })

        successful_corner_detections += 1
      else:
        failed_corner_detections += 1
        continue

print(f"Results written to {csv_file_path}")

# Print summary
total_scans = len(dirs)
print("\n\n--------CORNER DETECTION SUMMARY----------")
print(f"\nTotal number of images scanned: {total_scans}")
print(f"\nSuccessful corner detections: {successful_corner_detections}")
print(f"\nFailed corner detections: {failed_corner_detections}")
corner_detection_accuracy = (successful_corner_detections / total_scans) * 100
print(f"\nCorner detection accuracy: {corner_detection_accuracy:.2f}%")

dataset = pd.read_csv('/content/drive/MyDrive/nic_project/results/results.csv')
dataset

Enter the link to the images (e.g., /content/drive/MyDrive/nic_project/test_images): /content/drive/MyDrive/nic_project/test_images

0: 640x448 1 top_left, 1 top_right, 1 bottom_right, 1 bottom_left, 117.8ms
Speed: 3.7ms preprocess, 117.8ms inference, 0.9ms postprocess per image at shape (1, 3, 640, 448)

0: 480x640 1 id_number, 1 gender, 1 bod, 119.3ms
Speed: 1.2ms preprocess, 119.3ms inference, 0.7ms postprocess per image at shape (1, 3, 480, 640)
Loads checkpoint by http backend from path: https://download.openmmlab.com/mmocr/textrecog/satrn/satrn_shallow_5e_st_mj/satrn_shallow_5e_st_mj_20220915_152443-5fd04a4c.pth


Output()

The model and loaded state dict do not match exactly

unexpected key in source state_dict: data_preprocessor.mean, data_preprocessor.std



Loads checkpoint by http backend from path: https://download.openmmlab.com/mmocr/textrecog/satrn/satrn_shallow_5e_st_mj/satrn_shallow_5e_st_mj_20220915_152443-5fd04a4c.pth


Output()

...

Loads checkpoint by http backend from path: https://download.openmmlab.com/mmocr/textrecog/satrn/satrn_shallow_5e_st_mj/satrn_shallow_5e_st_mj_20220915_152443-5fd04a4c.pth


Output()

The model and loaded state dict do not match exactly

unexpected key in source state_dict: data_preprocessor.mean, data_preprocessor.std



Results written to /content/drive/MyDrive/nic_project/results/results.csv


--------CORNER DETECTION SUMMARY----------

Total number of images scanned: 57

Successful corner detections: 50

Failed corner detections: 7

Corner detection accuracy: 87.72%


,Image,ID Number,Name,Gender,Date of Birth
0,0222.jpg,194884210090,THUSAYA HEWA RATHNAWATHI,Female,1948/12/07
1,0245.jpg,200272600425,ALUTH ARACHCHIGE ARACHCHIGEDON. DON HIRUNI DIL...,Female,2002/08/13
2,0257.jpg,199266204016,ARULAMPALAM SUREJINI,Female,1992/06/10
3,0211.jpg,197515202401,BOLABOTUWA MESTHRIYAGE NILANTHA BOLABOTUWA,Male,1975/05/31
4,0246.jpg,200332611860,NIVITHIGALA PAHAMUNAGE PAHAMUNA PASINDU CHAMUD...,Male,2003/11/21
5,0271.jpg,197328103500,KOSWELPALA BADAPISSALAGE SARATH WIJETHILAKA,Male,1973/10/07
6,0250.jpg,198616703790,HEWA KOPARAGE ASANKA LAKMAL,Male,1986/06/15
7,0224.jpg,198633902365,MAHA MUTHUPELAGE THARINDU DUSHYANTHA FERNANDO,Male,1986/12/04
8,0259.jpg,199457200220,WIJESUNDARA APPUHAMILAGE DARSHIKA NIROSHANEE,Female,1994/03/12
9,0217.jpg,195327400843,GAMARALLAGE WICKRAMASINGHE,Male,1953/09/30
